### Employees Earning More Than Their Managers - Leetcode

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql.session import SparkSession

spark = SparkSession.builder.appName("emp_earning_more_than_managers").master("local[*]").getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions",4)

columns = ["id", "name", "salary", "managerid"]
data = [(1, "Joe", 70000, 3), (2, "Henry", 80000, 4), (3, "Sam", 60000, None), (4, "Max", 90000, None)]

emp_df = spark.createDataFrame(data, columns)

emp_df.show()

+---+-----+------+---------+
| id| name|salary|managerid|
+---+-----+------+---------+
|  1|  Joe| 70000|        3|
|  2|Henry| 80000|        4|
|  3|  Sam| 60000|     null|
|  4|  Max| 90000|     null|
+---+-----+------+---------+



In [4]:
# We have to self join this table as both the employee and the manager information are present in the same dataframe.
# This is hierarchical data.
from pyspark.sql.functions import col

joined_df = emp_df.alias("e1").join(emp_df.alias("e2"), col("e1.managerid") == col("e2.id"), "inner")

joined_df.show()

joined_df.where("e1.salary > e2.salary").select("e1.name").show()


+---+-----+------+---------+---+----+------+---------+
| id| name|salary|managerid| id|name|salary|managerid|
+---+-----+------+---------+---+----+------+---------+
|  2|Henry| 80000|        4|  4| Max| 90000|     null|
|  1|  Joe| 70000|        3|  3| Sam| 60000|     null|
+---+-----+------+---------+---+----+------+---------+

+----+
|name|
+----+
| Joe|
+----+

